In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import os
# os.environ["KERAS_BACKEND"] = "tensorflow"
# import keras; import keras.backend
# if keras.backend.backend() != 'tensorflow':
#     raise BaseException("This script uses other backend")
# else:
#     keras.backend.set_image_dim_ordering('th')
#     print("Backend ok")

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import keras
from keras.applications.vgg16 import VGG16
from keras.applications.resnet50 import ResNet50

from keras.models import Model
from keras.layers import Dense, Dropout, Flatten
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from tqdm import tqdm
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import cv2
from sklearn.linear_model import LogisticRegression
# print(os.listdir("../input"))
from sklearn.metrics import log_loss, accuracy_score
# Any results you write to the current directory are saved as output.

/home/paperspace/anaconda3/envs/fastai/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
!ls ~/data/dogBreed

labels.csv	sample_submission.csv	   test      train
labels.csv.zip	sample_submission.csv.zip  test.zip  train.zip


In [3]:
df_train = pd.read_csv("~/data/dogBreed/labels.csv")
df_test = pd.read_csv("~/data/dogBreed/sample_submission.csv")

In [4]:
df_train.head(10)

,id,breed
0,000bec180eb18c7604dcecc8fe0dba07,boston_bull
1,001513dfcb2ffafc82cccf4d8bbaba97,dingo
2,001cdf01b096e06d78e9e5112d419397,pekinese
3,00214f311d5d2247d5dfe4fe24b2303d,bluetick
4,0021f9ceb3235effd7fcde7f7538ed62,golden_retriever
5,002211c81b498ef88e1b40b9abf84e1d,bedlington_terrier
6,00290d3e1fdd27226ba27a8ce248ce85,bedlington_terrier
7,002a283a315af96eaea0e28e7163b21b,borzoi
8,003df8b8a8b05244b1d920bb6cf451f9,basenji
9,0042188c895a2f14ef64a918ed9c7b64,scottish_deerhound


In [5]:
targets_series = pd.Series(df_train["breed"])
one_hot = pd.get_dummies(targets_series, sparse=True)

In [6]:
one_hot_labels = np.asarray(one_hot)

In [7]:
one_hot_labels.shape

(10222, 120)

In [8]:
im_size = 224

In [9]:
x_train = []
y_train = []
# x_test = []

In [10]:
!ls ~/data/dogBreed/

labels.csv	sample_submission.csv	   test      train
labels.csv.zip	sample_submission.csv.zip  test.zip  train.zip


In [11]:
i = 0
for f, breed in tqdm(df_train.values):
    img = cv2.imread(r"/home/paperspace/data/dogBreed/train/{}.jpg".format(f))
    label = one_hot_labels[i]
    x_train.append(cv2.resize(img, (im_size, im_size)))
    y_train.append(label)
    i += 1

100%|██████████| 10222/10222 [00:27<00:00, 366.68it/s]


In [12]:
y_train_raw = np.array(y_train, np.uint8)
x_train_raw = np.array(x_train, np.float32)/255.
x_train = None

In [13]:
print(x_train_raw.shape)
print(y_train_raw.shape)
# print(x_test.shape)

(10222, 224, 224, 3)
(10222, 120)


In [14]:
num_class = y_train_raw.shape[1]

In [15]:
# X_train, X_valid, Y_train, Y_valid = train_test_split(x_train_raw, y_train_raw, test_size=0.3, random_state=1)
train_propotion = 0.7
rnd = np.random.random(x_train_raw.shape[0])
train_idx = rnd < train_propotion
valid_idx = rnd >= train_propotion


In [16]:
x_train_raw.size

1538697216

In [17]:
X_train, Y_train, X_valid, Y_valid = (x_train_raw[train_idx], y_train_raw[train_idx], 
                                      x_train_raw[valid_idx], y_train_raw[valid_idx])
x_train_raw = None
y_train_raw = None

In [21]:

base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(im_size, im_size, 3), pooling="avg")
train_model_op = base_model.predict(X_train, batch_size=32, verbose=1)
valid_model_op = base_model.predict(X_valid, batch_size=32, verbose=1)

logreg = LogisticRegression(multi_class='multinomial', solver='lbfgs')
logreg.fit(train_model_op, (Y_train * range(num_class)).sum(axis=1))
valid_probs = logreg.predict_proba(valid_model_op)
valid_preds = logreg.predict(valid_model_op)

print('Validation VGG LogLoss {}'.format(log_loss(Y_valid, valid_probs)))
print('Validation VGG Accuracy {}'.format(accuracy_score((Y_valid * range(num_class)).sum(axis=1), valid_preds)))

2959/2959 [==============================] - 16s 5ms/step
Validation VGG LogLoss 4.550298585569924
Validation VGG Accuracy 0.048327137546468404


In [22]:
valid_preds

array([ 75,  87,  95, ..., 118,  12,  28])

In [24]:
Y_valid

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8)

In [18]:
# Resnet50
base_model = ResNet50(weights="imagenet", include_top=False, input_shape=(im_size, im_size, 3))
x = base_model.output
x = Flatten()(x)
x = Dense(2048, activation="relu")(x)
x = Dense(1024, activation="relu")(x)
x = Dropout(0.5)(x)
predictions = Dense(num_class, activation="softmax")(x)

model = Model(inputs=base_model.input, outputs=predictions)
for layer in base_model.layers:
    layer.trainable = False

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
callbacks_list = [keras.callbacks.EarlyStopping(monitor="val_acc", patience=3, 
                                               verbose=1)]
# model.summary()

94658560/94653016 [==============================] - 114s 1us/step
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
__________________________________________

In [19]:
# Resnet50
model.fit(X_train, Y_train, epochs=25, 
          validation_data=(X_valid, Y_valid), verbose=1)

Train on 7174 samples, validate on 3048 samples
Epoch 1/25
7174/7174 [==============================] - 55s 8ms/step - loss: 2.9910 - acc: 0.2934 - val_loss: 5.0148 - val_acc: 0.0075
Epoch 2/25
7174/7174 [==============================] - 46s 6ms/step - loss: 1.5179 - acc: 0.5556 - val_loss: 5.3902 - val_acc: 0.0144
Epoch 3/25
7174/7174 [==============================] - 46s 6ms/step - loss: 1.0961 - acc: 0.6735 - val_loss: 5.6368 - val_acc: 0.0144
Epoch 4/25
7174/7174 [==============================] - 46s 6ms/step - loss: 0.8819 - acc: 0.7340 - val_loss: 5.4400 - val_acc: 0.0059
Epoch 5/25
7174/7174 [==============================] - 46s 6ms/step - loss: 0.7194 - acc: 0.7812 - val_loss: 5.7596 - val_acc: 0.0144
Epoch 6/25
7174/7174 [==============================] - 46s 6ms/step - loss: 0.5717 - acc: 0.8174 - val_loss: 5.6366 - val_acc: 0.0075
Epoch 7/25
7174/7174 [==============================] - 46s 6ms/step - loss: 0.5101 - acc: 0.8354 - val_loss: 6.0764 - val_acc: 0.0072
Epoch 8

KeyboardInterrupt: 

In [21]:
# VGG
base_model = VGG16(weights="imagenet", include_top=False, input_shape=(im_size, im_size, 3))
x = base_model.output
x = Flatten()(x)
x = Dense(2048, activation="relu")(x)
x = Dense(1024, activation="relu")(x)
x = Dropout(0.5)(x)
predictions = Dense(num_class, activation="softmax")(x)

model = Model(inputs=base_model.input, outputs=predictions)
for layer in base_model.layers:
    layer.trainable = False

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
callbacks_list = [keras.callbacks.EarlyStopping(monitor="val_acc", patience=3, 
                                               verbose=1)]
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [ ]:
# VGG
model.fit(X_train, Y_train, epochs=25, 
          validation_data=(X_valid, Y_valid), verbose=1)

Train on 7129 samples, validate on 3093 samples
Epoch 1/25
7129/7129 [==============================] - 65s 9ms/step - loss: 5.3403 - acc: 0.0189 - val_loss: 4.3851 - val_acc: 0.0411
Epoch 2/25
7129/7129 [==============================] - 61s 9ms/step - loss: 4.0067 - acc: 0.0835 - val_loss: 3.6423 - val_acc: 0.1254
Epoch 3/25
7129/7129 [==============================] - 61s 9ms/step - loss: 3.2160 - acc: 0.1846 - val_loss: 3.2961 - val_acc: 0.1820
Epoch 4/25
7129/7129 [==============================] - 61s 9ms/step - loss: 2.6343 - acc: 0.2857 - val_loss: 3.2774 - val_acc: 0.2014
Epoch 5/25
7129/7129 [==============================] - 61s 9ms/step - loss: 2.1449 - acc: 0.3860 - val_loss: 3.3332 - val_acc: 0.2098
Epoch 6/25
7129/7129 [==============================] - 61s 9ms/step - loss: 1.1295 - acc: 0.6476 - val_loss: 3.8210 - val_acc: 0.2270
Epoch 9/25
7129/7129 [==============================] - 61s 9ms/step - loss: 0.9223 - acc: 0.7106 - val_loss: 4.1771 - val_acc: 0.2182
Epoch 1